In [1]:
import pandas as pd
import numpy as np
import warnings; 
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')
import h2o
h2o.init(max_mem_size = "5G")
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from sklearn.base import BaseEstimator, TransformerMixin
from h2o.grid.grid_search import H2OGridSearch

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 days 20 hours 33 mins
H2O cluster version:,3.11.0.3714
H2O cluster version age:,"1 year, 5 months and 8 days !!!"
H2O cluster name:,H2O_from_python_yihsuanfu_xzr9v4
H2O cluster total nodes:,1
H2O cluster free memory:,0
H2O cluster total cores:,16
H2O cluster allowed cores:,16
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [2]:
flag_early = 0
if flag_early:
    tag_pred = 'early'
else:
    tag_pred = 'late'

In [3]:
# LOAD AND MERGE DATA FILES

# load
path = '/data/WorkData/spatialtemporal/'
file = pd.read_csv(path + 'finalmerge_any_' + tag_pred + 'v2.csv')
timefile = pd.read_csv(path + 'gr_lastyear_' + tag_pred + '.csv')
test_cases = pd.read_csv(path + 'test_cases_last_hearingv2.csv', header=None)

# merge
file = pd.merge(file, timefile, on=['idnproceeding','idncase'], how='left')

In [4]:
test_cases = test_cases.rename(columns={0:'num'})
train = file[~file.idncase.isin(test_cases.num)]
test = file[file.idncase.isin(test_cases.num)]

In [5]:
train.count()

idncase                            438696
idnproceeding                      438696
dec                                438696
numAppsPerProc                     438696
nat                                438696
case_type                          438696
c_asy_type                         438696
base_city_code                     438696
hearing_loc_code                   438696
osc_date                           438696
comp_date                          438696
attorney_flag                      438696
tracid                             438696
hearing_city                       438696
base_city                          438696
numProcPerCase                     438696
adj_medium                         438696
adj_date                           438696
sched_type                         438696
lang_hearing                       438696
notice_desc                        207300
adj_time_start2                    207300
adj_time_stop2                     207300
durationHearing                   

In [6]:
# columns to drop in transformer depending on the model
if flag_early: 
    cols = ['idncase', 'idnproceeding','adj_date','comp_date','osc_date','base_city_code', 'hearing_loc_code'
            'notice_desc','adj_time_start2','adj_time_stop2', 'caseDuration',
            'numAppsPerProc','numProcPerCase','osc_date','adj_rsn_desc', 'durationFirstLastHearing', 'averageHearingDur'
            'numHearingsPerProc']
else: 
    cols = ['idncase', 'idnproceeding','adj_date','comp_date','osc_date','numAppsPerProc','numProcPerCase','base_city_code',
            'hearing_loc_code', 'notice_desc','adj_time_start2','adj_time_stop2','adj_rsn_desc']
    



In [7]:
class Cleaning(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        # drop unused columns        
        X = X.drop(columns=cols)
        # changes frame to H2O compatible frame
        X = h2o.H2OFrame(X)
        # encodes as categorical
        X['dec'] = X['dec'].asfactor()
        return X
    

In [8]:
clean = Cleaning()
train = clean.transform(train)

H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Total input file size of 105.3 MB is much larger than total cluster memory of Zero  , please use either a larger cluster or smaller data.
  Request: POST /3/Parse
    data: {'destination_frame': 'Key_Frame__upload_b46dc5bc729908e02d08a11584044283.hex', 'parse_type': 'CSV', 'separator': '44', 'single_quotes': 'False', 'check_header': '1', 'number_columns': '32', 'chunk_size': '4194304', 'delete_on_done': 'True', 'blocking': 'False', 'column_types': '["Numeric","Enum","Enum","Enum","Numeric","Numeric","Enum","Enum","Enum","Enum","Enum","Numeric","Numeric","Numeric","Numeric","Enum","Numeric","Numeric","Enum","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric","Numeric"]', 'column_names': '["dec","nat","case_type","c_asy_type","attorney_flag","tracid","hearing_city","base_city","adj_medium","sched_type","lang_hearing","durationHearing","numHearing

In [38]:
train.shape

(438696, 32)

In [45]:
train.types

{'adj_medium': 'enum',
 'attorney_flag': 'int',
 'averageHearingDur': 'real',
 'base_city': 'enum',
 'base_city_code_dmy_last1yr_late': 'int',
 'base_city_code_gr_last1yr_late': 'real',
 'base_city_code_num_last1yr_late': 'int',
 'c_asy_type': 'enum',
 'caseDuration': 'int',
 'case_type': 'enum',
 'dec': 'enum',
 'durationFirstLastHearing': 'int',
 'durationHearing': 'real',
 'hearingDayOfWeek': 'enum',
 'hearingMonth': 'int',
 'hearingYear': 'int',
 'hearing_city': 'enum',
 'lang_hearing': 'enum',
 'nat': 'enum',
 'nat_dmy_last1yr_late': 'int',
 'nat_gr_last1yr_late': 'real',
 'nat_num_last1yr_late': 'int',
 'numHearingsPerProc': 'int',
 'osc_date_delta': 'int',
 'pres_aff': 'enum',
 'sched_type': 'enum',
 'tracid': 'int',
 'tracid_dmy_last10d': 'int',
 'tracid_dmy_last1yr_late': 'int',
 'tracid_gr_last10d': 'real',
 'tracid_gr_last1yr_late': 'real',
 'tracid_num_last1yr_late': 'int'}

In [65]:
train['attorney_flag'] = train['attorney_flag'].asfactor()
train['tracid'] = train['tracid'].asfactor()


In [66]:
train.types

{'adj_medium': 'enum',
 'attorney_flag': 'enum',
 'averageHearingDur': 'real',
 'base_city': 'enum',
 'base_city_code_dmy_last1yr_late': 'int',
 'base_city_code_gr_last1yr_late': 'real',
 'base_city_code_num_last1yr_late': 'int',
 'c_asy_type': 'enum',
 'caseDuration': 'int',
 'case_type': 'enum',
 'dec': 'enum',
 'durationFirstLastHearing': 'int',
 'durationHearing': 'real',
 'hearingDayOfWeek': 'enum',
 'hearingMonth': 'int',
 'hearingYear': 'int',
 'hearing_city': 'enum',
 'lang_hearing': 'enum',
 'nat': 'enum',
 'nat_dmy_last1yr_late': 'int',
 'nat_gr_last1yr_late': 'real',
 'nat_num_last1yr_late': 'int',
 'numHearingsPerProc': 'int',
 'osc_date_delta': 'int',
 'pres_aff': 'enum',
 'sched_type': 'enum',
 'tracid': 'enum',
 'tracid_dmy_last10d': 'int',
 'tracid_dmy_last1yr_late': 'int',
 'tracid_gr_last10d': 'real',
 'tracid_gr_last1yr_late': 'real',
 'tracid_num_last1yr_late': 'int'}

In [81]:
# specifying which columns to use as x and y 
#training_columns = train.columns.values.tolist()[1:] #EB commented out because it wasn't working
training_columns = train.columns[1:]
response_column = 'dec'

In [82]:
training_columns

['nat',
 'case_type',
 'c_asy_type',
 'attorney_flag',
 'tracid',
 'hearing_city',
 'base_city',
 'adj_medium',
 'sched_type',
 'lang_hearing',
 'durationHearing',
 'numHearingsPerProc',
 'durationFirstLastHearing',
 'averageHearingDur',
 'pres_aff',
 'osc_date_delta',
 'caseDuration',
 'hearingDayOfWeek',
 'hearingYear',
 'hearingMonth',
 'nat_num_last1yr_late',
 'nat_gr_last1yr_late',
 'nat_dmy_last1yr_late',
 'tracid_num_last1yr_late',
 'tracid_gr_last1yr_late',
 'tracid_dmy_last1yr_late',
 'base_city_code_num_last1yr_late',
 'base_city_code_gr_last1yr_late',
 'base_city_code_dmy_last1yr_late',
 'tracid_gr_last10d',
 'tracid_dmy_last10d']

In [83]:
model = H2ORandomForestEstimator(ntrees=100, max_depth=20, nfolds=5, 
                                 keep_cross_validation_predictions= True, stopping_metric='auc')
model.train(x=training_columns, y=response_column, training_frame=train)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [85]:
model

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1526163829723_999


ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.14878763817774474
RMSE: 0.3857300068412422
LogLoss: 0.46155033190961003
Mean Per-Class Error: 0.16966521009441238
AUC: 0.9071334429305073
Gini: 0.8142668858610147
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5928773543956307: 


,0,1,Error,Rate
0,216995.0,43979.0,0.1685,(43979.0/260974.0)
1,30357.0,147365.0,0.1708,(30357.0/177722.0)
Total,247352.0,191344.0,0.1694,(74336.0/438696.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.5928774,0.7985835,170.0
max f2,0.4423977,0.8550861,233.0
max f0point5,0.7185830,0.8163597,123.0
max accuracy,0.6424248,0.8351729,151.0
max precision,0.9907360,0.9937598,2.0
max recall,0.0058935,1.0,398.0
max specificity,0.9959670,0.9999962,0.0
max absolute_mcc,0.6350083,0.6568957,154.0
max min_per_class_accuracy,0.5909654,0.8301057,171.0
max mean_per_class_accuracy,0.5928774,0.8303348,170.0


Gains/Lift Table: Avg response rate: 40.51 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-05-16 14:49:00,9 min 23.154 sec,0.0,nan,nan,nan,nan,nan
,2018-05-16 14:49:01,9 min 23.887 sec,1.0,0.4941546,5.4642675,0.7345498,1.5098222,0.2995910
,2018-05-16 14:49:02,9 min 24.734 sec,2.0,0.4979228,5.2941802,0.7291742,1.8821358,0.3166198
,2018-05-16 14:49:03,9 min 25.744 sec,3.0,0.4822859,4.3600415,0.7503379,2.1759166,0.2842763
,2018-05-16 14:49:04,9 min 27.012 sec,4.0,0.4721095,3.6873916,0.7651225,2.3211279,0.2720545
,2018-05-16 14:49:10,9 min 32.691 sec,12.0,0.4191291,0.9683908,0.8476620,2.4678768,0.2269342
,2018-05-16 14:49:21,9 min 43.523 sec,27.0,0.3960582,0.4897888,0.8863328,2.4684395,0.1923911
,2018-05-16 14:49:38,10 min 1.252 sec,51.0,0.3896193,0.4682514,0.8998467,2.4678768,0.1746403
,2018-05-16 14:50:12,10 min 34.592 sec,97.0,0.3861280,0.4623092,0.9068761,2.4684395,0.1696870
,2018-05-16 14:50:20,10 min 42.536 sec,100.0,0.3857300,0.4615503,0.9071334,2.4684395,0.1694476


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
nat,904885.1250000,1.0,0.1655791
tracid,879589.3125000,0.9720453,0.1609504
nat_gr_last1yr_late,730698.6875000,0.8075044,0.1337058
tracid_gr_last1yr_late,604530.75,0.6680746,0.1106192
base_city_code_gr_last1yr_late,274885.3125000,0.3037792,0.0502995
---,---,---,---
tracid_dmy_last10d,16926.0351562,0.0187052,0.0030972
pres_aff,9799.0966797,0.0108291,0.0017931
tracid_dmy_last1yr_late,541.2819214,0.0005982,0.0000990
nat_dmy_last1yr_late,532.6488037,0.0005886,0.0000975



See the whole table with table.as_data_frame()


In [98]:
#cross validation
model = H2ORandomForestEstimator(max_depth=20, nfolds=5, 
                                keep_cross_validation_predictions= True, stopping_metric='auc')
#model = H2ORandomForestEstimator(ntrees=100, nfolds=5, 
 #                                keep_cross_validation_predictions= True, stopping_metric='auc')
param_grid = {'ntrees': [100,200,400,800]}
#param_grid = {'max_depth': [5,10,20,40]}

grid_search = H2OGridSearch(model, param_grid)
grid_search.train(x=training_columns, y=response_column, training_frame=train)


drf Grid Build progress: |████████████████████████████████████████████████| 100%


ValueError: Gridsearch returns no model due to bad parameter values or other reasons....

In [92]:
grid_search.get_grid

    max_depth  \
0           5   

                                                         model_ids  \
0  Grid_DRF_py_35_sid_9bb7_model_python_1526163829723_1745_model_0   

             logloss  
0  0.484805210252584  


<bound method H2OGridSearch.get_grid of >

In [93]:
result = grid_search.get_grid(sort_by='auc', decreasing=True)
result

    max_depth  \
0           5   

                                                         model_ids  \
0  Grid_DRF_py_35_sid_9bb7_model_python_1526163829723_1745_model_0   

                 auc  
0  0.847297263901737  


In [ ]:
best_estimator = result.models[0]


In [80]:
model._model_json['output']['variable_importances'].as_data_frame()

,variable,relative_importance,scaled_importance,percentage
0,tracid,932755.750000,1.000000,0.171461
1,nat,868487.437500,0.931098,0.159647
2,nat_gr_last1yr_late,711172.000000,0.762442,0.130729
3,tracid_gr_last1yr_late,573431.125000,0.614771,0.105409
4,lang_hearing,292596.906250,0.313691,0.053786
5,base_city_code_gr_last1yr_late,235325.015625,0.252290,0.043258
6,tracid_gr_last10d,216251.015625,0.231841,0.039752
7,hearing_city,185548.093750,0.198925,0.034108
8,base_city,171619.187500,0.183992,0.031547
9,osc_date_delta,122460.210938,0.131289,0.022511
